<!-- Desenvolvido por Silmara Basso- silmarabasso@yahoo.com.br -->
# <font color='blue'>Classificação de animais</font>
# Pinguins.
 <font color='blue'>Deploy com SageMaker, Lambda e API Gateway</font>


In [1]:
# Imports
import os
import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Configurações Iniciais no SageMaker

In [2]:
# Define a região
aws_region = boto3.Session().region_name
print(aws_region)

us-east-2


In [3]:
# Define a role
aws_role = get_execution_role()
print(aws_role)

arn:aws:iam::626216981464:role/service-role/AmazonSageMaker-ExecutionRole-20250730T193400


In [4]:
# Cria sessão SageMaker
aws_session = sagemaker.Session()
print(aws_session)

## Carregando os Dados

In [5]:
# Cria o cliente s3
s3 = boto3.client("s3")

In [ ]:
# Download do arquivo do bucket para o ambiente local do SageMaker
s3.download_file(f"pinguins-890582101704", "dataset.csv", "dados_originais.csv")

In [ ]:
# Carrega o arquivo como dataframe do pandas
df_pinguins = pd.read_csv("./dados_originais.csv")

## Exploração Inicial dos Dados

In [9]:
# Visualiza as primeiras linhas
df_pinguins.head()

,id,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [10]:
# Shape
df_pinguins.shape

(344, 9)

In [11]:
# Total de valores ausentes
df_pinguins.isna().sum()

id                    0
species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
year                  0
dtype: int64

In [12]:
# Total de duplicatas (complete cases)
df_pinguins.duplicated().sum()

0

In [13]:
# Info
df_pinguins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 344 non-null    int64  
 1   species            344 non-null    object 
 2   island             344 non-null    object 
 3   bill_length_mm     342 non-null    float64
 4   bill_depth_mm      342 non-null    float64
 5   flipper_length_mm  342 non-null    float64
 6   body_mass_g        342 non-null    float64
 7   sex                333 non-null    object 
 8   year               344 non-null    int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 24.3+ KB


In [14]:
# Total de valores únicos
df_pinguins.nunique()

id                   344
species                3
island                 3
bill_length_mm       164
bill_depth_mm         80
flipper_length_mm     55
body_mass_g           94
sex                    2
year                   3
dtype: int64

In [15]:
# Resumo estatístico
df_pinguins.describe()

,id,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
count,344.000000,342.000000,342.000000,342.000000,342.000000,344.000000
mean,171.500000,43.921930,17.151170,200.915205,4201.754386,2008.031977
std,99.448479,5.459584,1.974793,14.061714,801.954536,0.816464
min,0.000000,32.100000,13.100000,172.000000,2700.000000,2007.000000
25%,85.750000,39.225000,15.600000,190.000000,3550.000000,2007.000000
50%,171.500000,44.450000,17.300000,197.000000,4050.000000,2008.000000
75%,257.250000,48.500000,18.700000,213.000000,4750.000000,2009.000000
max,343.000000,59.600000,21.500000,231.000000,6300.000000,2009.000000


## Limpeza, Codificação e Preparação dos Dados

In [16]:
# Colunas
df_pinguins.columns

Index(['id', 'species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex', 'year'],
      dtype='object')

In [17]:
print("Categorias na Variável 'species': ", df_pinguins['species'].unique())

Categorias na Variável 'species':  ['Adelie' 'Gentoo' 'Chinstrap']


In [18]:
print("Categorias na Variável 'island': ", df_pinguins['island'].unique())

Categorias na Variável 'island':  ['Torgersen' 'Biscoe' 'Dream']


In [19]:
dados_limpos = df_pinguins.drop(columns = ['id', 'island', 'sex', 'year'])

In [20]:
# Info
dados_limpos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   bill_length_mm     342 non-null    float64
 2   bill_depth_mm      342 non-null    float64
 3   flipper_length_mm  342 non-null    float64
 4   body_mass_g        342 non-null    float64
dtypes: float64(4), object(1)
memory usage: 13.6+ KB


In [21]:
# Vamos fazer o drop de valores ausentes
dados_limpos = dados_limpos.dropna(subset = ['bill_length_mm'])

In [22]:
dados_limpos.isna().sum()

species              0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
dtype: int64

In [23]:
dados_limpos.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,39.1,18.7,181.0,3750.0
1,Adelie,39.5,17.4,186.0,3800.0
2,Adelie,40.3,18.0,195.0,3250.0
4,Adelie,36.7,19.3,193.0,3450.0
5,Adelie,39.3,20.6,190.0,3650.0


In [24]:
# Label encoding na variável alvo
dados_limpos['species'] = dados_limpos['species'].replace('Adelie', 0)
dados_limpos['species'] = dados_limpos['species'].replace('Gentoo', 1)
dados_limpos['species'] = dados_limpos['species'].replace('Chinstrap', 2)

In [25]:
# Shuffle (embaralhamento dos dados)
dados_limpos = dados_limpos.sample(frac = 1).reset_index(drop = True)

In [26]:
dados_limpos.sample(10)

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
206,0,41.1,18.6,189.0,3325.0
89,1,47.2,15.5,215.0,4975.0
186,0,40.6,18.8,193.0,3800.0
74,0,40.8,18.9,208.0,4300.0
151,1,46.7,15.3,219.0,5200.0
28,1,45.0,15.4,220.0,5050.0
7,1,49.1,14.8,220.0,5150.0
24,0,43.2,18.5,192.0,4100.0
187,2,49.8,17.3,198.0,3675.0
34,2,40.9,16.6,187.0,3200.0


## Divisão dos Dados em Treino e Teste e Armazenamento no S3

In [27]:
# Garante que somente as colunas do shape do dataframe serão usadas 
dados_limpos.columns = range(dados_limpos.shape[1])

In [28]:
dados_limpos.sample(5)

,0,1,2,3,4
138,2,50.5,19.6,201.0,4050.0
232,0,41.3,20.3,194.0,3550.0
166,0,38.1,18.6,190.0,3700.0
42,2,51.0,18.8,203.0,4100.0
334,0,39.7,17.7,193.0,3200.0


In [29]:
# Divisão em treino e validação com proporção 80/20
pinguins_dados_treino, pinguins_dados_valid = train_test_split(dados_limpos, test_size = 0.2, random_state = 42)

In [30]:
pinguins_dados_treino.shape

(273, 5)

In [31]:
pinguins_dados_valid.shape

(69, 5)

In [32]:
# Nome do bucket S3
bucket_name = 'pinguins-890582101704'

In [34]:
# Salvando e enviando o arquivo de treino
pinguins_dados_treino.to_csv('dados_treino.csv', header = False, index = False)
key = 'pinguins/treino/dados_treino'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('dados_treino.csv')

In [35]:
# Salvando e enviando o arquivo de validação
pinguins_dados_valid.to_csv('dados_valid.csv', header = False, index = False)
key = 'pinguins/val/dados_valid'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('dados_valid.csv')

## Construção do Modelo

In [36]:
# Nome do bucket S3
bucket_name = 'pinguins-890582101704'

In [37]:
# Nome para salvar o modelo
key = 'pinguins_model/xgbmodel'

In [38]:
# Local no S3 para salvar o modelo
s3_output_location = f's3://{bucket_name}/{key}'

In [40]:
# Define a URI da imagem do modelo XGBoost no SageMaker
image_uri = sagemaker.image_uris.retrieve("xgboost", aws_region, version = "1.5-1")

In [41]:
# Cria o estimador XGBoost
pinguins_xgbmodel = sagemaker.estimator.Estimator(image_uri = image_uri,
                                             role = aws_role,
                                             instance_count = 1,
                                             instance_type = 'ml.m4.xlarge',
                                             volume_size = 5,
                                             output_path = s3_output_location,
                                             sagemaker_session = aws_session)

In [42]:
# Configura os hiperparâmetros
pinguins_xgbmodel.set_hyperparameters(max_depth = 5,
                                 gamma = 4,
                                 min_child_weight = 6,
                                 objective = 'multi:softmax',
                                 num_class = 3,
                                 num_round = 8)

Descrição dos hiperparâmetros configurados no modelo pinguins_xgbmodel para o algoritmo XGBoost:

**max_depth (5)**: Define a profundidade máxima de cada árvore de decisão no modelo. Controla a complexidade do modelo, evitando que ele se torne excessivamente complexo e, portanto, sujeito a overfitting. Um valor de 5 permite que cada árvore tenha até 5 níveis de profundidade.

**gamma (4)**: Representa o valor mínimo de redução na função de perda para que uma nova divisão em uma árvore seja feita. Um valor maior torna o modelo mais conservador, pois exige maior melhora na função objetivo para criar novas divisões.

**min_child_weight (6)**: Especifica o peso mínimo de somatório de instâncias em um nó filho para que ele seja subdividido. Este parâmetro ajuda a controlar o underfitting. Um valor maior exige que cada nó tenha mais instâncias, resultando em árvores mais simples.

**objective ('multi:softmax')**: Define a função objetivo para o modelo. Aqui, foi configurado como multi:softmax, que é usado para problemas de classificação multiclasse. Ele gera como saída a classe prevista diretamente, em vez de probabilidades.

**num_class (3)**: Indica o número de classes distintas no problema de classificação. Nesse caso, o modelo está configurado para lidar com 3 classes diferentes.
**num_round (8)**: Refere-se ao número de iterações ou rodadas de boosting. Cada rodada adiciona uma nova árvore ao ensemble para melhorar as previsões do modelo. Com 8 rodadas, o modelo será atualizado iterativamente até esse limite.

## Treinamento do Modelo

In [43]:
# Define o caminho para os dados
dados_treino_pinguins = 's3://{}/{}'.format(bucket_name, 'pinguins/treino/dados_treino')
dados_valid_pinguins = 's3://{}/{}'.format(bucket_name, 'pinguins/val/dados_val')

In [44]:
dados_treino_pinguins

's3://pinguins-890582101704/pinguins/treino/dados_treino'

In [45]:
dados_valid_pinguins

's3://pinguins-890582101704/pinguins/val/dados_val'

In [46]:
# Cria os inputs
input_treino = TrainingInput(s3_data = dados_treino_pinguins, content_type = 'text/csv')
input_val = TrainingInput(s3_data = dados_valid_pinguins, content_type = 'text/csv')

In [47]:
# Define os canais para os dados
canais_dados = {'train': input_treino, 'validation': input_val}

In [48]:
%%time

pinguins_xgbmodel.fit(inputs = canais_dados)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-07-30-23-49-04-149


2025-07-30 23:49:05 Starting - Starting the training job...
2025-07-30 23:49:18 Starting - Preparing the instances for training...
2025-07-30 23:49:44 Downloading - Downloading input data...
2025-07-30 23:50:14 Downloading - Downloading the training image...
2025-07-30 23:51:09 Training - Training image download completed. Training in progress....
2025-07-30 23:51:30 Uploading - Uploading generated training model/miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-07-30 23:51:25.597 ip-10-0-157-84.us-east-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-07-30 23:51:25.620 ip-10-0-157-84.us-east-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-07-30:23:51:25:INFO] Imported framework sagemaker_xgboost_conta

## Deploy do Modelo com SageMaker

In [49]:
%%time

xgb_predictor = pinguins_xgbmodel.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-07-30-23-57-07-172
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-07-30-23-57-07-172
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-07-30-23-57-07-172


------!CPU times: user 50.6 ms, sys: 12.8 ms, total: 63.4 ms
Wall time: 3min 32s


Ao executar a linha acima, o modelo treinado (pinguins_xgbmodel) é implantado em um endpoint gerenciado pelo Amazon SageMaker para inferência em tempo real. Aqui está o que acontece:

**Criação do Endpoint**: O SageMaker cria um endpoint RESTful onde o modelo ficará disponível para receber solicitações de previsão.

**Configuração da Infraestrutura**: É provisionada uma instância do tipo ml.m4.xlarge para hospedar o modelo. Essa instância é dimensionada com base nos requisitos computacionais do modelo.

**Inicialização do Modelo**: O modelo treinado é carregado na instância provisionada, junto com o ambiente necessário para sua execução, como o container do XGBoost.

**Atribuição ao Objeto xgb_predictor**: O objeto xgb_predictor é criado e serve como uma interface para interagir com o endpoint. Ele permite enviar dados para predição e receber as respostas diretamente no ambiente de desenvolvimento.

Este processo disponibiliza o modelo em um ambiente escalável e gerenciado, pronto para realizar inferências em tempo real a partir de requisições enviadas ao endpoint.

Agora vamos criar a API.

# Fim